<a href="https://colab.research.google.com/github/cemgurbey/sleep-log-data-analysis/blob/main/Sleep_Log_Data_Analysis_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sleep Log Data Analysis Tool

## How to use it

1. Click on the files icon on the left, then click on the upload icon to upload sleeplog data.
2. Please check the name of the files that you uploaded contains the words *morning* and *evening*.
3. Click on `Runtime` on the menu bar and click `Run all`.
4. Click on the refresh button on the Files tab and the desired output files will appear. You can download them by right clicking on the files.

Notes: 
- The files will be deleted on everytime you close the browser so make sure to save the output file on your device.
- If you Add new questions to the surveys or change the outline of the spreadsheet, please adjust the column letters down.

In [ ]:
MORNING_PARTICIPANT_NUMBER_COLUMN_LETTER = 'A'
EVENING_PARTICIPANT_NUMBER_COLUMN_LETTER = 'D'
MORNING_DATE_COLUMN_LETTER = 'B'
EVENING_DATE_COLUMN_LETTER = 'E'
BEDTIME_COLUMN_LETTER = 'H'
WOKETIME_COLUMN_LETTER = 'I'
OUT_BED_COLUMN_LETTER = 'J'
IN_BED_COLUMN_LETTER = 'F'
START_SCHOOL_COLUMN_LETTER = 'F'
FINISH_SCHOOL_COLUMN_LETTER = 'H'
GET_HOME_COLUMN_LETTER = 'J'


## Software and Explanations



### 1. Import necessary libraries for the software

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
from statistics import stdev
from os import listdir
from google.colab import drive, files

### 2. Adjust settings:


- Display all the columns
- Use Google Drive (Optional)
- Retrieve morning and evening data filenames

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# drive.mount('/content/drive')

In [ ]:
# gets the full file names from directory
for filename in listdir():
    if "morning" in filename.lower():
        MORNING_DATA_FILENAME = filename
    if "evening" in filename.lower():
        EVENING_DATA_FILENAME = filename

### 3. Optional Google Drive Integration or Manual Entry of the Filename

In [ ]:
# WRITE YOUR FILENAMES HERE
# MORNING_DATA_FILENAME = '1. SleepLog_MORNING_Winter2022_912_Group1_ORGANIZED_18Feb2022_DV.xlsx'  
# EVENING_DATA_FILENAME = '1. SleepLog_EVENING_Winter2022_912_Group1_ORGANIZED_18Feb2022_DV.xlsx'

# If you are using Google Drive, enter the full path of the filename after /content/drive/My Drive/
# MORNING_DATA_FILENAME = '/content/drive/My Drive/Douglas Research/1. SleepLog_MORNING_Winter2022_912_Group1_ORGANIZED_18Feb2022_DV.xlsx'
# EVENING_DATA_FILENAME = '/content/drive/My Drive/Douglas Research/1. SleepLog_EVENING_Winter2022_912_Group1_ORGANIZED_18Feb2022_DV.xlsx'

### 4. TODO

In [ ]:
def excel_column_number(name):
    """Excel-style column name to number, e.g., A = 1, Z = 26, AA = 27, AAA = 703."""
    n = 0
    for c in name:
        n = n * 26 + ord(c) - ord('A')
    return n

In [ ]:
# Calculates length of an event relative to 12:00 PM
def time_diff(x):
    if datetime.combine(date.min, x) > datetime.combine(date.min, dt.time(12)):
        return pd.to_timedelta(datetime.combine(date.min, x) - datetime.combine(date.min, dt.time(12)))
    
    return datetime.combine(date.min, x) - datetime.combine(date.min, dt.time(0)) + dt.timedelta(hours=12)

In [ ]:
def add_fields(df, columns, keyword):
    total = 0
    tranposed_df = df.iloc[:, columns].transpose()
    transposed_df.dropna(axis=1, inplace=True)

    for day in transposed_df.columns:
        total += transposed_df[transposed_df[day] == keyword].shape[0]
    return total


In [ ]:
def incosistent_time_format_helper(time_str):
    if type(time_str) == type("str"):
        if time_str[6:8].upper() in ['AM', 'PM']:
            if int(time_str[:2]) > 12:
                return datetime.strptime(time_str[:5],'%H:%M').time()
            else:
                return datetime.strptime(time_str,'%I:%M %p').time()



    return np.datetime64('NaT')


In [ ]:
MORNING_PARTICIPANT_NUMBER_COLUMN = excel_column_number(MORNING_PARTICIPANT_NUMBER_COLUMN_LETTER)
EVENING_PARTICIPANT_NUMBER_COLUMN = excel_column_number(EVENING_PARTICIPANT_NUMBER_COLUMN_LETTER)
BEDTIME_COLUMN = excel_column_number(BEDTIME_COLUMN_LETTER)
WOKETIME_COLUMN = excel_column_number(WOKETIME_COLUMN_LETTER)
OUT_BED_COLUMN = excel_column_number(OUT_BED_COLUMN_LETTER)
IN_BED_COLUMN = excel_column_number(IN_BED_COLUMN_LETTER)
START_SCHOOL_COLUMN = excel_column_number(START_SCHOOL_COLUMN_LETTER)
FINISH_SCHOOL_COLUMN = excel_column_number(FINISH_SCHOOL_COLUMN_LETTER)
GET_HOME_COLUMN = excel_column_number(GET_HOME_COLUMN_LETTER)
MORNING_DATE_COLUMN = excel_column_number(MORNING_DATE_COLUMN_LETTER)
EVENING_DATE_COLUMN = excel_column_number(EVENING_DATE_COLUMN_LETTER)

In [ ]:
# This section reads the excel sheet and preprocesses headers and time data

df_morning = pd.read_excel(MORNING_DATA_FILENAME, header=[0,1])
df_evening = pd.read_excel(EVENING_DATA_FILENAME, header=[0,1])

# Remove the unnamed headers
df_morning.columns = df_morning.columns.map(lambda x: ' '.join(x) if 'Unnamed' not in x[1] else x[0])
df_evening.columns = df_evening.columns.map(lambda x: ' '.join(x) if 'Unnamed' not in x[1] else x[0])

# Fill the empty lines
df_morning[df_morning.columns[MORNING_PARTICIPANT_NUMBER_COLUMN]] = df_morning[df_morning.columns[MORNING_PARTICIPANT_NUMBER_COLUMN]].fillna(0.0).astype(int)
df_evening[df_evening.columns[EVENING_PARTICIPANT_NUMBER_COLUMN]] = df_evening[df_evening.columns[EVENING_PARTICIPANT_NUMBER_COLUMN]].fillna(0.0).astype(int)

# Drop empty columns (requires re-indexing)
# df_morning.dropna(axis=1, how='all', inplace=True) 
# df_evening.dropna(axis=1, how='all', inplace=True) 

# The remaining of this block converts AM/PM time format to 24h format
# Incorrect time signatures like 16:30 PM will be ignored
df_morning[df_morning.columns[[START_SCHOOL_COLUMN, FINISH_SCHOOL_COLUMN,GET_HOME_COLUMN]]] = df_morning[df_morning.columns[[START_SCHOOL_COLUMN, FINISH_SCHOOL_COLUMN,GET_HOME_COLUMN]]]\
                                                                                                  .apply(lambda x: x.apply(lambda y: incosistent_time_format_helper(y)))
                                                                                                  

df_evening[df_evening.columns[[IN_BED_COLUMN, BEDTIME_COLUMN]]] = df_evening[df_evening.columns[[IN_BED_COLUMN, BEDTIME_COLUMN]]].apply(lambda x: pd.to_datetime(x.astype(str)).dt.time)
# ignore PMs for awake time and out bed time questions (ask this part to Gabrielle)
df_evening[df_evening.columns[[WOKETIME_COLUMN, OUT_BED_COLUMN]]] = df_evening[df_evening.columns[[WOKETIME_COLUMN, OUT_BED_COLUMN]]].astype('string').apply(lambda x:\
                                                             x.apply(lambda y: datetime.strptime(y[:5],'%H:%M').time()\
                                                             if type(y) == type("str") else np.datetime64('NaT')))

In [ ]:
grp_morning = df_morning.groupby(df_morning.columns[3]) #group by participant number of morning data
grp_evening = df_evening.groupby(df_evening.columns[3]) #group by participant number of evening data

In [ ]:
columns = ['partcipant_number']
mock_output = pd.DataFrame(columns=columns)

#iterate through each participant
for participant_number, df in grp_morning:

    if participant_number == 0: # skip the rows that spaces out the participants
        continue


    df.dropna(axis=0, subset=[df.columns[0]], inplace=True)
    temp_df = pd.DataFrame()
    temp_df['Date'] = pd.to_datetime(df[df.columns[4]], format="%d/%m/%Y")
    temp_df['Day_Of_Week'] = temp_df['Date'].dt.dayofweek

    SL_pre_weekdays_selfreport = temp_df[temp_df['Day_Of_Week'] < 4].shape[0]
    SL_pre_weekends_selfreport = temp_df[temp_df['Day_Of_Week'] >= 4].shape[0]

    light_exposure_total = 0
    transposed_df = df.iloc[:, list(range(95,107))].transpose()
    transposed_df.dropna(axis=1, inplace=True)

    # for day in transposed_df.columns:
    #     light_exposure_total += transposed_df[transposed_df[day] == 'LIGHTS TURNED ON'].shape[0]

    light_exposure_total = add_fields(df, list(range(95, 107)), 'LIGHTS TURNED ON')

    Average_lights_off_time_weeknights = temp_df[temp_df['Day_Of_Week'] < 4]
    
    light_exposure_avg = 0
    if transposed_df.shape[1] != 0:
        light_exposure_avg = light_exposure_total / transposed_df.shape[1]

    # append the results of a participant to the output dataframe
    mock_output.loc[mock_output.shape[0]] = [participant_number]


In [ ]:
columns = [
    'partcipant_number',
    'SL_pre_BEDTIME_AVERAGE_WEEK_selfreport',
    'SL_pre_WOKETIME_AVERAGE_WEEK_selfreport',
    'SL_pre_BEDTIME_AVERAGE_WEEKEND_selfreport',
    'SL_pre_WOKETIME_AVERAGE_WEEKEND_selfreport',
    'SL_pre_BEDTIME_AVERAGE_TOTAL_selfreport',
    'SL_pre_WOKETIME_AVERAGE_TOTAL_selfreport',
    'SL_pre_BEDTIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_WOKETIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_BEDTIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_WOKETIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_BEDTIME_SD_WEEK_selfreport',
    'SL_pre_WOKETIME_SD_WEEK_selfreport',
    'SL_pre_BEDTIME_SD_WEEKEND_selfreport',
    'SL_pre_WOKETIME_SD_WEEKEND_selfreport',
    'SL_pre_BEDTIME_SD_TOTAL_selfreport',
    'SL_pre_WOKETIME_SD_TOTAL_selfreport',
    'SL_pre_BEDTIME_SD_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_WOKETIME_SD_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_BEDTIME_SD_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_WOKETIME_SD_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_IN_BED_AVERAGE_WEEK_selfreport',
    'SL_pre_OUT_BED_AVERAGE_WEEK_selfreport',
    'SL_pre_IN_BED_AVERAGE_WEEKEND_selfreport',
    'SL_pre_OUT_BED_AVERAGE_WEEKEND_selfreport',
    'SL_pre_IN_BED_AVERAGE_TOTAL_selfreport',
    'SL_pre_OUT_BED_AVERAGE_TOTAL_selfreport',
    'SL_pre_IN_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_OUT_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_IN_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_OUT_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_IN_BED_SD_WEEK_selfreport',
    'SL_pre_OUT_BED_SD_WEEK_selfreport',
    'SL_pre_IN_BED_SD_WEEKEND_selfreport',
    'SL_pre_OUT_BED_SD_WEEKEND_selfreport',
    'SL_pre_IN_BED_SD_TOTAL_selfreport',
    'SL_pre_OUT_BED_SD_TOTAL_selfreport',
    'SL_pre_IN_BED_SD_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_OUT_BED_SD_WEEKWITHSUNDNIGHT_selfreport',
    'SL_pre_IN_BED_SD_WEEKENDWOSUNDNIGHT_selfreport',
    'SL_pre_OUT_BED_SD_WEEKENDWOSUNDNIGHT_selfreport',
]
mock_output = pd.DataFrame(columns=columns)
mini_output = pd.DataFrame(columns=['partcipant_number', 'day', 'timedelta'])

#iterate through each participant
for participant_number, df in grp_evening:

    if participant_number == 0: # skip the rows that spaces out the participants
        continue


    df.dropna(axis=0, subset=[df.columns[0]], inplace=True)

    df['Date'] = pd.to_datetime(df[df.columns[EVENING_DATE_COLUMN]].astype('string'), format="%d/%m/%Y")
    df['Day_Of_Week'] = df['Date'].dt.dayofweek
    SL_pre_weekdays_selfreport = df[df['Day_Of_Week'] < 4].shape[0]
    SL_pre_weekends_selfreport = df[df['Day_Of_Week'] >= 4].shape[0]

    light_exposure_total = 0
    transposed_df = df.iloc[:, list(range(95,107))].transpose()
    transposed_df.dropna(axis=1, inplace=True)

    SL_pre_BEDTIME_AVERAGE_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[7]].apply(time_diff).mean())[11:19]

    SL_pre_WOKETIME_AVERAGE_WEEK_selfreport = str(pd.to_datetime(df[df['Day_Of_Week'] < 4][df.columns[8]], format=('%H:%M:%S')).mean())[11:19]
    
    SL_pre_BEDTIME_AVERAGE_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                        + df[df['Day_Of_Week'] >= 4][df.columns[7]].apply(time_diff).mean())[11:19];

    SL_pre_WOKETIME_AVERAGE_WEEKEND_selfreport = str(pd.to_datetime(df[df['Day_Of_Week'] >= 4][df.columns[8]], format=('%H:%M:%S')).mean())[11:19];

    SL_pre_BEDTIME_AVERAGE_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[7]].apply(time_diff).mean())[11:19];

    SL_pre_WOKETIME_AVERAGE_TOTAL_selfreport = str(pd.to_datetime(df[df.columns[8]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_BEDTIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[7]].apply(time_diff).mean())[11:19]

    SL_pre_WOKETIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport = str(pd.to_datetime(df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)] \
                                                                              [df.columns[8]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_BEDTIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                    + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)]
                                                                        [df.columns[7]].apply(time_diff).mean())[11:19]

    SL_pre_WOKETIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport = str(pd.to_datetime(df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)] \
                                                                              [df.columns[8]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_BEDTIME_SD_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[7]].apply(time_diff).std() - dt.timedelta(hours=12) )[11:19]

    SL_pre_WOKETIME_SD_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[8]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_BEDTIME_SD_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                        + df[df['Day_Of_Week'] >= 4][df.columns[7]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_WOKETIME_SD_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                        + df[df['Day_Of_Week'] >= 4][df.columns[8]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_BEDTIME_SD_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[7]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_WOKETIME_SD_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[8]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_BEDTIME_SD_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[7]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_WOKETIME_SD_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[8]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_BEDTIME_SD_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)][df.columns[7]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_WOKETIME_SD_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)][df.columns[8]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_IN_BED_AVERAGE_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[5]].apply(time_diff).mean())[11:19]

    SL_pre_OUT_BED_AVERAGE_WEEK_selfreport = str(pd.to_datetime(df[df['Day_Of_Week'] < 4][df.columns[9]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_IN_BED_AVERAGE_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                        + df[df['Day_Of_Week'] >= 4][df.columns[5]].apply(time_diff).mean())[11:19]

    SL_pre_OUT_BED_AVERAGE_WEEKEND_selfreport = str(pd.to_datetime(df[df['Day_Of_Week'] >= 4][df.columns[9]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_IN_BED_AVERAGE_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[5]].apply(time_diff).mean())[11:19]

    SL_pre_OUT_BED_AVERAGE_TOTAL_selfreport = str(pd.to_datetime(df[df.columns[9]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_IN_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[5]].apply(time_diff).mean())[11:19]

    SL_pre_OUT_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport = str(pd.to_datetime(df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)] \
                                                                              [df.columns[9]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_IN_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                    + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)]
                                                                        [df.columns[5]].apply(time_diff).mean())[11:19]

    SL_pre_OUT_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport = str(pd.to_datetime(df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)] \
                                                                              [df.columns[9]], format=('%H:%M:%S')).mean())[11:19]

    SL_pre_IN_BED_SD_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[5]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_OUT_BED_SD_WEEK_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] < 4][df.columns[9]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_IN_BED_SD_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] >= 4][df.columns[5]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_OUT_BED_SD_WEEKEND_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df['Day_Of_Week'] >= 4][df.columns[9]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_IN_BED_SD_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[5]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_OUT_BED_SD_TOTAL_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                    + df[df.columns[9]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_IN_BED_SD_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[5]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_OUT_BED_SD_WEEKWITHSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] < 4) | (df['Day_Of_Week'] == 6)][df.columns[9]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_IN_BED_SD_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)][df.columns[5]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    SL_pre_OUT_BED_SD_WEEKENDWOSUNDNIGHT_selfreport = str(datetime.combine(date.min, dt.time(12)) \
                                                                + df[(df['Day_Of_Week'] >= 4) & (df['Day_Of_Week'] != 6)][df.columns[9]].apply(time_diff).std() - dt.timedelta(hours=12))[11:19]

    light_exposure_avg = 0
    if transposed_df.shape[1] != 0:
        light_exposure_avg = light_exposure_total / transposed_df.shape[1]

    # append the results of a participant to the output dataframe
    mock_output.loc[mock_output.shape[0]] = [
        participant_number, 
        SL_pre_BEDTIME_AVERAGE_WEEK_selfreport,
        SL_pre_WOKETIME_AVERAGE_WEEK_selfreport,
        SL_pre_BEDTIME_AVERAGE_WEEKEND_selfreport,
        SL_pre_WOKETIME_AVERAGE_WEEKEND_selfreport,
        SL_pre_BEDTIME_AVERAGE_TOTAL_selfreport,
        SL_pre_WOKETIME_AVERAGE_TOTAL_selfreport,
        SL_pre_BEDTIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_WOKETIME_AVERAGE_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_BEDTIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_WOKETIME_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_BEDTIME_SD_WEEK_selfreport,
        SL_pre_WOKETIME_SD_WEEK_selfreport,
        SL_pre_BEDTIME_SD_WEEKEND_selfreport,
        SL_pre_WOKETIME_SD_WEEKEND_selfreport,
        SL_pre_BEDTIME_SD_TOTAL_selfreport,
        SL_pre_WOKETIME_SD_TOTAL_selfreport,
        SL_pre_BEDTIME_SD_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_WOKETIME_SD_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_BEDTIME_SD_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_WOKETIME_SD_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_IN_BED_AVERAGE_WEEK_selfreport,
        SL_pre_OUT_BED_AVERAGE_WEEK_selfreport,
        SL_pre_IN_BED_AVERAGE_WEEKEND_selfreport,
        SL_pre_OUT_BED_AVERAGE_WEEKEND_selfreport,
        SL_pre_IN_BED_AVERAGE_TOTAL_selfreport,
        SL_pre_OUT_BED_AVERAGE_TOTAL_selfreport,
        SL_pre_IN_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_OUT_BED_AVERAGE_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_IN_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_OUT_BED_AVERAGE_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_IN_BED_SD_WEEK_selfreport,
        SL_pre_OUT_BED_SD_WEEK_selfreport,
        SL_pre_IN_BED_SD_WEEKEND_selfreport,
        SL_pre_OUT_BED_SD_WEEKEND_selfreport,
        SL_pre_IN_BED_SD_TOTAL_selfreport,
        SL_pre_OUT_BED_SD_TOTAL_selfreport,
        SL_pre_IN_BED_SD_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_OUT_BED_SD_WEEKWITHSUNDNIGHT_selfreport,
        SL_pre_IN_BED_SD_WEEKENDWOSUNDNIGHT_selfreport,
        SL_pre_OUT_BED_SD_WEEKENDWOSUNDNIGHT_selfreport,
    ]


ValueError: ignored

In [ ]:
df[df.columns[[EVENING_PARTICIPANT_NUMBER_COLUMN, EVENING_DATE_COLUMN]]]

,Please enter the participant number that was sent to you here so that we can identify you: Open-Ended Response,What was yesterday's date? (i.e. the date of your bedtime) Day/Month/Year
80,5909,2022-08-02 00:00:00
81,5909,2022-09-02 00:00:00
82,5909,2022-10-02 00:00:00
83,5909,2022-11-02 00:00:00
84,5909,2022-12-02 00:00:00
85,5909,13/02/2022
86,5909,14/02/2022


In [ ]:
mock_output

REMAINING TASKS

1) sum total lights off time for each day (col [96-106])

2) sum total woke up time per day

4) 

**col1** Average lights off time (weeknights excluding Sunday night) - M, T, W, Th	
**col2** Average woke up time (weekdays excluding Sunday Morning) - M, T, W, Th	
col3 Average lights off time (weekends including Sunday Nights) - F, Sat, Sun	
col4 Average woke up time (weekend days including Sunday mornings) - F, Sat, Sun
col5 Average lights off time (whole week)	Average woke up time (whole week)	
col6 Average lights off time (weeknights including Sunday nights) - M, T, W, Th, Sun
col7 Average woke up time (weekdays including Sunday Morning) - M, T, W, Th, Sun
col8 Average lights off time (weekends excluding Sunday Nights) - F, Sat
col9 Average woke up time (weekend days excluding Sunday mornings) - F, Sat


####slx column names

In [ ]:
slx_daytime_column_names = [ 
 'SL_pre_weekdays_selfreport',
 'SL_pre_weekends_selfreport',
 'SLx_pre_sunrise_time',
 'SLx_pre_sunset_time',
 'SLx_pre_daylength',
 'SLx_pre_sunlight',
 'SLx_pre_moonphase',
 'SLx_pre_moonlight',
 'SLx_pre_Date_start',
 'SLx_pre_Date_end',
 'SLx_pre_Day',
 'SLx_pre_BEDTIME_selfreport',
 'SLx_pre_LIGHTSOFF_selfreport',
 'SLx_pre_NAP_selfreport',
 'SLx_pre_NAPTIME_selfreport',
 'SLx_pre_NAP_START_selfreport',
 'SLx_pre_NAP_END_selfreport',
 'SLx_pre_NAPTIME2_selfreport',
 'SLx_pre_NAP2_START_selfreport',
 'SLx_pre_NAP2_END_selfreport',
 'SLx_pre_NAP_other_selfreport',
 'SLx_pre_ACTIVITIES_tv_selfreport',
 'SLx_pre_ACTIVITIES_tv_duration',
 'SLx_pre_ACTIVITIES_internet_selfreport',
 'SLx_pre_ACTIVITIES_internet_duration',
 'SLx_pre_ACTIVITIES_phone_selfreport',
 'SLx_pre_ACTIVITIES_phone_duration',
 'SLx_pre_ACTIVITIES_videogames_selfreport',
 'SLx_pre_ACTIVITIES_videogames_duration',
 'SLx_pre_ACTIVITIES_read_selfreport',
 'SLx_pre_ACTIVITIES_read_duration',
 'SLx_pre_ACTIVITIES_music_selfreport',
 'SLx_pre_ACTIVITIES_music_duration',
 'SLx_pre_ACTIVITIES_other_selfreport',
 'SLx_pre_ACTIVITIES_other_duration',
 'SLx_PRE_ACTIVITIES_other2_selfreport',
 'SLx_PRE_ACTIVITIES_other2_duration',
 'SLx_pre_WOKETIME_selfreport',
 'SLx_pre_GETUP_selfreport',
 'SLx_pre_SCHOOL_ATTEND',
 'SLx_pre_SCHOOL_START',
 'SLx_pre_SCHOOL_END',
 'SLx_pre_HOME_ARRIVAL',
 'SLx_pre_PHONE_12_1am',
 'SLx_pre_COMPUTER_12_1am',
 'SLx_pre_TABLET_12_1am',
 'SLx_pre_VIDGAMES_12_1am',
 'SLx_pre_PHONE_1_2am',
 'SLx_pre_COMPUTER_1_2am',
 'SLx_pre_TABLET_1_2am',
 'SLx_pre_VIDGAMES_1_2am',
 'SLx_pre_PHONE_2_3am',
 'SLx_pre_COMPUTER_2_3am',
 'SLx_pre_TABLET_2_3am',
 'SLx_pre_VIDGAMES_2_3am',
 'SLx_pre_PHONE_3_4am',
 'SLx_pre_COMPUTER_3_4am',
 'SLx_pre_TABLET_3_4am',
 'SLx_pre_VIDGAMES_3_4am',
 'SLx_pre_PHONE_4_5am',
 'SLx_pre_COMPUTER_4_5am',
 'SLx_pre_TABLET_4_5am',
 'SLx_pre_VIDGAMES_4_5am',
 'SLx_pre_PHONE_5_6am',
 'SLx_pre_COMPUTER_5_6am',
 'SLx_pre_TABLET_5_6am',
 'SLx_pre_VIDGAMES_5_6am',
 'SLx_pre_PHONE_6_7am',
 'SLx_pre_COMPUTER_6_7am',
 'SLx_pre_TABLET_6_7am',
 'SLx_pre_VIDGAMES_6_7am',
 'SLx_pre_PHONE_7_8am',
 'SLx_pre_COMPUTER_7_8am',
 'SLx_pre_TABLET_7_8am',
 'SLx_pre_VIDGAMES_7_8am',
 'SLx_pre_PHONE_8_9am',
 'SLx_pre_COMPUTER_8_9am',
 'SLx_pre_TABLET_8_9am',
 'SLx_pre_VIDGAMES_8_9am',
 'SLx_pre_PHONE_9_10am',
 'SLx_pre_COMPUTER_9_10am',
 'SLx_pre_TABLET_9_10am',
 'SLx_pre_VIDGAMES_9_10am',
 'SLx_pre_PHONE_10_11am',
 'SLx_pre_COMPUTER_10_11am',
 'SLx_pre_TABLET_10_11am',
 'SLx_pre_VIDGAMES_10_11am',
 'SLx_pre_PHONE_11_12pm',
 'SLx_pre_COMPUTER_11_12pm',
 'SLx_pre_TABLET_11_12pm',
 'SLx_pre_VIDGAMES_11_12pm',
 'SLx_pre_PHONE_12_1pm',
 'SLx_pre_COMPUTER_12_1pm',
 'SLx_pre_TABLET_12_1pm',
 'SLx_pre_VIDGAMES_12_1pm',
 'SLx_pre_PHONE_1_2pm',
 'SLx_pre_COMPUTER_1_2pm',
 'SLx_pre_TABLET_1_2pm',
 'SLx_pre_VIDGAMES_1_2pm',
 'SLx_pre_PHONE_2_3pm',
 'SLx_pre_COMPUTER_2_3pm',
 'SLx_pre_TABLET_2_3pm',
 'SLx_pre_VIDGAMES_2_3pm',
 'SLx_pre_PHONE_3_4pm',
 'SLx_pre_COMPUTER_3_4pm',
 'SLx_pre_TABLET_3_4pm',
 'SLx_pre_VIDGAMES_3_4pm',
 'SLx_pre_PHONE_4_5pm',
 'SLx_pre_COMPUTER_4_5pm',
 'SLx_pre_TABLET_4_5pm',
 'SLx_pre_VIDGAMES_4_5pm',
 'SLx_pre_PHONE_5_6pm',
 'SLx_pre_COMPUTER_5_6pm',
 'SLx_pre_TABLET_5_6pm',
 'SLx_pre_VIDGAMES_5_6pm',
 'SLx_pre_PHONE_6_7pm',
 'SLx_pre_COMPUTER_6_7pm',
 'SLx_pre_TABLET_6_7pm',
 'SLx_pre_VIDGAMES_6_7pm',
 'SLx_pre_PHONE_7_8pm',
 'SLx_pre_COMPUTER_7_8pm',
 'SLx_pre_TABLET_7_8pm',
 'SLx_pre_VIDGAMES_7_8pm',
 'SLx_pre_PHONE_8_9pm',
 'SLx_pre_COMPUTER_8_9pm',
 'SLx_pre_TABLET_8_9pm',
 'SLx_pre_VIDGAMES_8_9pm',
 'SLx_pre_PHONE_9_10pm',
 'SLx_pre_COMPUTER_9_10pm',
 'SLx_pre_TABLET_9_10pm',
 'SLx_pre_VIDGAMES_9_10pm',
 'SLx_pre_PHONE_10_11pm',
 'SLx_pre_COMPUTER_10_11pm',
 'SLx_pre_TABLET_10_11pm',
 'SLx_pre_VIDGAMES_10_11pm',
 'SLx_pre_PHONE_11_12am',
 'SLx_pre_COMPUTER_11_12am',
 'SLx_pre_TABLET_11_12am',
 'SLx_pre_VIDGAMES_11_12am',
 'SLx_pre_INDOORS_12_1am',
 'SLx_pre_INDOORS_1_2am',
 'SLx_pre_INDOORS_2_3am',
 'SLx_pre_INDOORS_3_4am',
 'SLx_pre_INDOORS_4_5am',
 'SLx_pre_INDOORS_5_6am',
 'SLx_pre_INDOORS_6_7am',
 'SLx_pre_INDOORS_7_8am',
 'SLx_pre_INDOORS_8_9am',
 'SLx_pre_INDOORS_9_10am',
 'SLx_pre_INDOORS_10_11am',
 'SLx_pre_INDOORS_11_12pm',
 'SLx_pre_INDOORS_12_1pm',
 'SLx_pre_INDOORS_1_2pm',
 'SLx_pre_INDOORS_2_3pm',
 'SLx_pre_INDOORS_3_4pm',
 'SLx_pre_INDOORS_4_5pm',
 'SLx_pre_INDOORS_5_6pm',
 'SLx_pre_INDOORS_6_7pm',
 'SLx_pre_INDOORS_7_8pm',
 'SLx_pre_INDOORS_8_9pm',
 'SLx_pre_INDOORS_9_10pm',
 'SLx_pre_INDOORS_10_11pm',
 'SLx_pre_INDOORS_11_12am',
 'SLx_pre_OUTDOORS_6_7am',
 'SLx_pre_OUTDOORS_7_8am',
 'SLx_pre_OUTDOORS_8_9am',
 'SLx_pre_OUTDOORS_9_10am',
 'SLx_pre_OUTDOORS_10_11am',
 'SLx_pre_OUTDOORS_11_12pm',
 'SLx_pre_OUTDOORS_12_1pm',
 'SLx_pre_OUTDOORS_1_2pm',
 'SLx_pre_OUTDOORS_2_3pm',
 'SLx_pre_OUTDOORS_3_4pm',
 'SLx_pre_OUTDOORS_4_5pm',
 'SLx_pre_OUTDOORS_5_6pm',
 'SLx_pre_OUTDOORS_6_7pm',
 'SLx_pre_OUTDOORS_7_8pm',
 'SLx_pre_OUTDOORS_8_9pm' ]    

In [ ]:
slx_nighttime_column_names=[
 'SLx_PRE_ACTIVITIES_lightsoff',
 'SLx_PRE_ACTIVITIES_tv_pleasure',
 'SLx_PRE_ACTIVITIES_tv_arousal',
 'SLx_PRE_ACTIVITIES_tv_type_comedy',
 'SLx_PRE_ACTIVITIES_tv_type_horror',
 'SLx_PRE_ACTIVITIES_tv_type_romance',
 'SLx_PRE_ACTIVITIES_tv_type_action',
 'SLx_PRE_ACTIVITIES_tv_type_thriller',
 'SLx_PRE_ACTIVITIES_tv_type_fantasy',
 'SLx_PRE_ACTIVITIES_tv_type_mystery',
 'SLx_PRE_ACTIVITIES_tv_type_drama',
 'SLx_PRE_ACTIVITIES_tv_type_other',
 'SLx_PRE_ACTIVITIES_tv_type_other_specify',
 'SLx_PRE_ACTIVITIES_internet_pleasure',
 'SLx_PRE_ACTIVITIES_internet_arousal',
 'SLx_PRE_ACTIVITIES_internet_platform',
 'SLx_PRE_ACTIVITIES_phone_pleasure',
 'SLx_PRE_ACTIVITIES_phone_arousal',
 'SLx_PRE_ACTIVITIES_videogames_pleasure',
 'SLx_PRE_ACTIVITIES_videogames_arousal',
 'SLx_PRE_ACTIVITIES_read_pleasure',
 'SLx_PRE_ACTIVITIES_read_arousal',
 'SLx_PRE_ACTIVITIES_read_type_textbook',
 'SLx_PRE_ACTIVITIES_read_type_horror',
 'SLx_PRE_ACTIVITIES_read_romance',
 'SLx_PRE_ACTIVITIES_read_type_action',
 'SLx_PRE_ACTIVITIES_read_type_thriller',
 'SLx_PRE_ACTIVITIES_read_type_fantasy',
 'SLx_PRE_ACTIVITIES_read_type_mystery',
 'SLx_PRE_ACTIVITIES_read_type_drama',
 'SLx_PRE_ACTIVITIES_read_type_other',
 'SLx_PRE_ACTIVITIES_read_type_other_specify',
 'SLx_PRE_ACTIVITIES_read_format',
 'SLx_PRE_ACTIVITIES_music_type',
 'SLx_PRE_ACTIVITIES_music_pleasure',
 'SLx_PRE_ACTIVITIES_music_arousal',
 'SLx_PRE_ACTIVITIES_other_pleasure',
 'SLx_PRE_ACTIVITIES_other_arousal',
 'SLx_PRE_ACTIVITIES_other2_pleasure',
 'SLx_PRE_ACTIVITIES_other2_arousal',
 'SLx_PRE_ACTIVITIES_SleepApp',
 'SLx_PRE_ACTIVITIES_SleepApp_name',
 'SLx_PRE_ACTIVITIES_SleepApp_use_sounds',
 'SLx_PRE_ACTIVITIES_SleepApp_use_stories',
 'SLx_PRE_ACTIVITIES_SleepApp_use_relax',
 'SLx_PRE_ACTIVITIES_SleepApp_use_midfulness',
 'SLx_PRE_ACTIVITIES_SleepApp_use_meditation',
 'SLx_PRE_ACTIVITIES_SleepApp_use_other',
 'SLx_PRE_ACTIVITIES_SleepApp_use_specify',
 'SLx_PRE_ACTIVITIES_SleepApp_duration',
 'SLx_PRE_ACTIVITIES_SleepApp_helped',
 'SLx_PRE_OTHER_12_1am',
 'SLx_PRE_OTHER_1_2am',
 'SLx_PRE_OTHER_2_3am',
 'SLx_PRE_OTHER_3_4am',
 'SLx_PRE_OTHER_4_5am',
 'SLx_PRE_OTHER_5_6am',
 'SLx_PRE_OTHER_12_6am_specify',
 'SLx_PRE_OTHER_6_7am',
 'SLx_PRE_OTHER_7_8am',
 'SLx_PRE_OTHER_8_9am',
 'SLx_PRE_OTHER_9_10am',
 'SLx_PRE_OTHER_10_11am',
 'SLx_PRE_OTHER_11_12pm',
 'SLx_PRE_OTHER_6_12pm_specify',
 'SLx_PRE_OTHER_12_1pm',
 'SLx_PRE_OTHER_1_2pm',
 'SLx_PRE_OTHER_2_3pm',
 'SLx_PRE_OTHER_3_4pm',
 'SLx_PRE_OTHER_4_5pm',
 'SLx_PRE_OTHER_5_6pm',
 'SLx_PRE_OTHER_12_6pm_specify',
 'SLx_PRE_OTHER_6_7pm',
 'SLx_PRE_OTHER_7_8pm',
 'SLx_PRE_OTHER_8_9pm',
 'SLx_PRE_OTHER_9_10pm',
 'SLx_PRE_OTHER_10_11pm',
 'SLx_PRE_OTHER_11_12am',
 'SLx_PRE_OTHER_6_12am_specify',
 'SLx_PRE_AWL_wear',
 'SLx_PRE_AWL_on',
 'SLx_PRE_AWL_off',
 'SLx_PRE_AWL_other',
 'SLx_PRE_AWL_time_1',
 'SLx_PRE_AWL_duration_1',
 'SLx_PRE_AWL_time_2',
 'SLx_PRE_AWL_duration_2',
 'SLx_PRE_AWL_time_3',
 'SLx_PRE_AWL_duration_3',
 'SLx_PRE_AWL_time_4',
 'SLx_PRE_AWL_duration_4',
 'SLx_PRE_POMS_A_1',
 'SLx_PRE_POMS_A_2',
 'SLx_PRE_POMS_A_3',
 'SLx_PRE_POMS_A_4',
 'SLx_PRE_POMS_A_5',
 'SLx_PRE_POMS_A_6',
 'SLx_PRE_POMS_A_7',
 'SLx_PRE_POMS_A_8',
 'SLx_PRE_POMS_A_9',
 'SLx_PRE_POMS_A_10',
 'SLx_PRE_POMS_A_11',
 'SLx_PRE_POMS_A_12',
 'SLx_PRE_POMS_A_13',
 'SLx_PRE_POMS_A_14',
 'SLx_PRE_POMS_A_15',
 'SLx_PRE_POMS_A_16',
 'SLx_PRE_POMS_A_17',
 'SLx_PRE_POMS_A_18',
 'SLx_PRE_POMS_A_19',
 'SLx_PRE_POMS_A_20',
 'SLx_PRE_POMS_A_21',
 'SLx_PRE_POMS_A_22',
 'SLx_PRE_POMS_A_23',
 'SLx_PRE_POMS_A_24',
 'SLx_PRE_POMS_A_anger',
 'SLx_PRE_POMS_A_confusion',
 'SLx_PRE_POMS_A_depression',
 'SLx_PRE_POMS_A_fatigue',
 'SLx_PRE_POMS_A_tension',
 'SLx_PRE_POMS_A_vigour',
 'SLx_PRE_POMS_A_total'
]

In [ ]:
for i in range(10):
    continue

### 5. Output

Here is an example of how how the results will look like

In [ ]:
mock_output


In [ ]:
writer = pd.ExcelWriter('AVERAGE_AND_SD_OUTPUTS.xlsx')
mock_output.to_excel(writer, index=False)
writer.save()

# files.download('mock_output.xlsx')